In [140]:
import requests
import re
import pandas

In [141]:
class Datastore():
    def __init__(self,ckan_url, api_key, project_id):
        self.ckan_url = ckan_url
        self.api_key = api_key
        self.project_id = project_id

    def search_sql(self, name_query):
        with requests.Session() as session:
            params = {"id": project_id}
            resp = session.get(f"{self.ckan_url}/api/3/action/package_show", params=params)
            if resp.status_code == 200:
                uuid_mapping = {resource["name"]: resource["id"] for resource in resp.json()["result"]["resources"]}
            else: raise Exception

            # Replace resource names in name_query with uuids.
            # https://stackoverflow.com/questions/2400504/easiest-way-to-replace-a-string-using-a-dictionary-of-replacements
            pattern = re.compile(r'\b(' + '|'.join(uuid_mapping.keys()) + r')\b')
            uuid_query = pattern.sub(lambda x: uuid_mapping[x.group()], name_query)
            params = {"sql": uuid_query}
            resp = session.get(f"{self.ckan_url}/api/3/action/datastore_search_sql", params=params)
            if resp.status_code == 200:
                return pandas.DataFrame(resp.json()["result"]["records"])
            else: raise Exception
            

In [143]:
datastore = Datastore(ckan_url="http://127.0.0.1:5000", api_key=None, project_id="jupyter_test_project")

In [158]:
datastore.search_sql('SELECT * FROM "iris.csv" where petal_length > 2')

,petal_width,sepal_length,_full_text,petal_length,sepal_width,_id,species
0,1.4,7.0,'1.4':4 '3.2':2 '4.7':3 '7.0':1 'versicolor':5,4.7,3.2,51,versicolor
1,1.5,6.4,'1.5':4 '3.2':2 '4.5':3 '6.4':1 'versicolor':5,4.5,3.2,52,versicolor
2,1.5,6.9,'1.5':4 '3.1':2 '4.9':3 '6.9':1 'versicolor':5,4.9,3.1,53,versicolor
3,1.3,5.5,'1.3':4 '2.3':2 '4.0':3 '5.5':1 'versicolor':5,4.0,2.3,54,versicolor
4,1.5,6.5,'1.5':4 '2.8':2 '4.6':3 '6.5':1 'versicolor':5,4.6,2.8,55,versicolor
...,...,...,...,...,...,...,...
95,2.3,6.7,'2.3':4 '3.0':2 '5.2':3 '6.7':1 'virginica':5,5.2,3.0,146,virginica
96,1.9,6.3,'1.9':4 '2.5':2 '5.0':3 '6.3':1 'virginica':5,5.0,2.5,147,virginica
97,2.0,6.5,'2.0':4 '3.0':2 '5.2':3 '6.5':1 'virginica':5,5.2,3.0,148,virginica
98,2.3,6.2,'2.3':4 '3.4':2 '5.4':3 '6.2':1 'virginica':5,5.4,3.4,149,virginica
